Kesci competition

### 导入数据

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train_raw = pd.read_csv('train_set.csv')

#data
test_raw = pd.read_csv('test_set.csv')

#test_data
print(train_raw.dtypes)

ImportError: No module named 'lightgbm'

查看数据的指标

In [ ]:
train_raw.describe()

数据预处理

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

for col in train_raw.columns[train_raw.dtypes=='object']:
    le = LabelEncoder()
    le.fit(train_raw[col])
    
    train_raw[col] = le.transform(train_raw[col])
    test_raw[col] = le.transform(test_raw[col])
    
    
#print(train_raw)
#print(test_raw)

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

scaler.fit(train_raw[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']])

train_raw[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']]=scaler.transform(train_raw[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']])
test_raw[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']]=scaler.transform(test_raw[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']])

train_raw.head(100)
test_raw.head(100)

分割训练集和验证集

In [ ]:
from sklearn.model_selection import train_test_split

prediction = train_raw['y']

features = train_raw.drop('y', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(features, prediction, test_size=0.2, random_state=42)

print(features)

print("Training and testing split was successful.")

使用lgb进行训练

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 5,
    'max_depth': 6,
    'min_data_in_leaf': 450,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5,
    'lambda_l1': 1,  
    'lambda_l2': 0.001,
    'min_gain_to_split': 0.2,
    'verbose': 5,
    'is_unbalance': True
}

gbm = lgb.train(params,
               lgb_train,
               num_boost_round=1000,
               valid_sets=lgb_eval,
               valid_names=None,
               fobj=None, feval=None, init_model=None,
                feature_name='auto', categorical_feature='auto',
                early_stopping_rounds=50, evals_result=None,
                  verbose_eval=True,
                  keep_training_booster=False, callbacks=None)

preds = gbm.predict(test_raw, num_iteration=gbm.best_iteration)

# print(preds)

# threshold = 0.5

# for pred in preds:
#     result = 1 if pred > threshold else 0
    
test_raw['y'] = preds
test_raw.head(10000)

提交结果

In [ ]:
# submit_frame = pd.read_csv("submit.csv")

# result = pd.merge(submit_frame, test_raw, on='ID', how='left')
test_raw[['ID', 'y']].to_csv("result.csv",index=False)